In [16]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

from IPython.display import display
%matplotlib inline

In [17]:
# Read student data
loan_data = pd.read_csv("loan.csv", low_memory=False)
print ("Loan data read successfully!")

Loan data read successfully!


In [18]:
print (loan_data.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599     5000.0       5000.0           4975.0   36 months   
1  1077430    1314167     2500.0       2500.0           2500.0   60 months   
2  1077175    1313524     2400.0       2400.0           2400.0   36 months   
3  1076863    1277178    10000.0      10000.0          10000.0   36 months   
4  1075358    1311748     3000.0       3000.0           3000.0   60 months   

   int_rate  installment grade sub_grade     ...      total_bal_il il_util  \
0     10.65       162.87     B        B2     ...               NaN     NaN   
1     15.27        59.83     C        C4     ...               NaN     NaN   
2     15.96        84.33     C        C5     ...               NaN     NaN   
3     13.49       339.31     C        C1     ...               NaN     NaN   
4     12.69        67.79     B        B5     ...               NaN     NaN   

  open_rv_12m  open_rv_24m max_bal_bc all_util total_rev_hi_li

In [46]:
loan_outcome_data = loan_data.drop(loan_data[(loan_data.loan_status != 'Fully Paid') & (loan_data.loan_status != 'Charged Off')].index)
#display(loan_outcome_data.head())

#pif = (loan_outcome_data[loan_outcome_data['loan_status'] == 'Fully Paid'].count()['id'])
#chg = (loan_outcome_data[loan_outcome_data['loan_status'] == 'Charged Off'].count()['id']) 
#print('There are %.0f loans that paid in full, and %.0f loans that were charged off' % (pif, chg))

In [52]:
# TODO: Calculate number of records
n_loans = float(len(loan_outcome_data))

# TODO: Calculate number of features
n_features = float(len(loan_outcome_data.columns) - 1)

# TODO: Calculate paid in full loans
n_pif = float(sum(1 for item in loan_outcome_data['loan_status'] if item==('Fully Paid')))

# TODO: Calculate charged off loans
n_chargeoff = float(sum(1 for item in loan_outcome_data['loan_status'] if item==('Charged Off')))

# TODO: Calculate successful loan rate
success_rate = n_pif / n_loans * 100

# Print the results
print("Total number of loans: {}".format(n_loans))
print("Number of features: {}".format(n_features))
print("Number of loans paid in full: {}".format(n_pif))
print("Number of charge-offs: {}".format(n_chargeoff))
print("Success rate of loans: {:.2f}%".format(success_rate))

Total number of loans: 252971.0
Number of features: 73.0
Number of loans paid in full: 207723.0
Number of charge-offs: 45248.0
Success rate of loans: 82.11%


In [54]:
# Store the 'loan_status' feature in a new variable and remove it from the dataset
outcomes = loan_outcome_data['loan_status']
data = loan_outcome_data.drop(['loan_status', 'id', 'member_id', 'url', 'zip_code'], axis = 1)

# Show the new dataset with 'loan_status' removed
display(data.head())
display(outcomes.head())

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,Veolia Transportaton,3 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0     Fully Paid
1    Charged Off
2     Fully Paid
3     Fully Paid
5     Fully Paid
Name: loan_status, dtype: object

In [55]:
display(data.describe())

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,252971.000000,252971.000000,252971.000000,252971.000000,252971.000000,2.529710e+05,252971.000000,252971.000000,252971.000000,112151.000000,...,144.000000,126.000000,144.000000,144.000000,144.000000,144.000000,1.892630e+05,144.000000,144.000000,144.000000
mean,13562.768855,13522.193749,13429.174497,13.764625,418.112216,7.253825e+04,16.544104,0.249914,0.852485,35.175531,...,36462.312500,73.657937,1.673611,3.555556,5517.340278,58.945139,2.970912e+04,1.256944,2.097222,2.625000
std,8131.729531,8109.211388,8127.682956,4.401926,244.933059,5.881181e+04,7.788696,0.740600,1.066875,21.802850,...,38137.563875,23.074812,1.629406,2.932284,4507.604776,20.356957,2.951730e+04,1.792551,3.903234,4.057067
min,500.000000,500.000000,0.000000,5.320000,15.690000,3.000000e+03,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.900000,0.000000e+00,0.000000,0.000000,-4.000000
25%,7200.000000,7200.000000,7200.000000,10.740000,239.550000,4.500000e+04,10.750000,0.000000,0.000000,17.000000,...,10763.750000,63.350000,0.000000,2.000000,2091.250000,46.825000,1.330000e+04,0.000000,0.000000,1.000000
50%,12000.000000,12000.000000,11975.000000,13.530000,365.230000,6.200000e+04,16.200000,0.000000,1.000000,32.000000,...,23605.000000,77.050000,1.000000,3.000000,4511.500000,61.950000,2.230000e+04,1.000000,0.000000,2.000000
75%,18250.000000,18075.000000,18000.000000,16.550000,547.550000,8.700000e+04,21.990000,0.000000,1.000000,51.000000,...,50835.750000,88.250000,2.000000,5.000000,8212.750000,74.225000,3.680000e+04,1.000000,2.000000,4.000000
max,35000.000000,35000.000000,35000.000000,28.990000,1424.570000,8.706582e+06,57.140000,29.000000,8.000000,152.000000,...,249212.000000,129.200000,8.000000,21.000000,22279.000000,102.800000,2.013133e+06,9.000000,21.000000,19.000000
